# Lecture 3 – Data 100, Summer 2025

Data 100, Summer 2025

[Acknowledgments Page](https://ds100.org/su25/acks/)

A demonstration of advanced `pandas` syntax to accompany Lecture 3.

In [ ]:
import numpy as np
import pandas as pd

## Dataset: California baby names

In today's lecture, we'll work with the `babynames` dataset, which contains information about the names of infants born in California.

The cell below pulls census data from a government website and then loads it into a usable form. The code shown here is outside of the scope of Data 100, but you're encouraged to dig into it if you are interested!

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

## Conditional Selection

In [ ]:
# Ask yourself: Why is :9 is the correct slice to select the first 10 rows?
# Is this the correct slice if we used context-based extraction?
# What about .iloc? Try it!
babynames_first_10_rows = babynames.loc[:9, :]

babynames_first_10_rows

By passing in a sequence (list, array, or `Series`) of boolean values, we can extract a subset of the rows in a `DataFrame`. We will keep *only* the rows that correspond to a boolean value of `True`.

In [ ]:
# Notice how we have exactly 10 elements in our boolean array argument.
babynames_first_10_rows[[True, False, True, False, True, False, True, False, True, False]]

In [ ]:
# Or using .loc to filter a DataFrame by a Boolean array argument.
babynames_first_10_rows.loc[[True, False, True, False, True, False, True, False, True, False], :]

Oftentimes, we'll use boolean selection to check for entries in a `DataFrame` that meet a particular condition.

In [ ]:
# First, use a logical condition to generate a boolean Series
logical_operator = (babynames["Sex"]=="F")
logical_operator

In [ ]:
# Then, use this boolean Series to filter the DataFrame
babynames[logical_operator]

Boolean selection also works with `loc`!

In [ ]:
# Notice that we did not have to specify columns to select 
# If no columns are referenced, pandas will automatically select all columns
babynames.loc[babynames["Sex"]=="F"]

To filter on multiple conditions, we combine boolean operators using **bitwise comparisons**.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

In [ ]:
babynames[(babynames["Sex"]=="F") & (babynames["Year"]<2000)]

In [ ]:
babynames[(babynames["Sex"]=="F") | (babynames["Year"]<2000)]

## Slido Exercise
We want to obtain the first three baby names with `count > 250`.

<img src="images/slido_1.png" width="200"/>

In [ ]:
babynames.iloc[[0, 233, 484], [3, 4]]

In [ ]:
babynames.loc[[0, 233, 484]]

In [ ]:
babynames.loc[babynames["Count"] > 250, ["Name", "Count"]].head(3)

In [ ]:
babynames.loc[babynames["Count"] > 250, ["Name", "Count"]].iloc[0:2, :]

In [ ]:
# Note: The parentheses surrounding the code make it possible to 
# break the code into multiple lines for readability

(
    babynames[(babynames["Name"]=="Bella") | 
              (babynames["Name"]=="Alex") |
              (babynames["Name"]=="Narges") |
              (babynames["Name"]=="Lisa")]
)


In [ ]:
# A more concise method to achieve the above: .isin
names = ["Bella", "Alex", "Narges", "Lisa"]
display(babynames["Name"].isin(names))
display(babynames[babynames["Name"].isin(names)])

In [ ]:
# What if we only want names that start with "N"?
display(babynames["Name"].str.startswith("N"))

display(babynames[babynames["Name"].str.startswith("N")])

## Adding, Removing, and Modifying Columns

To add a column, use `[]` to reference the desired new column, then assign it to a `Series` or array of appropriate length.

In [ ]:
# Create a Series of the length of each name.
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name.
babynames["name_lengths"] = babyname_lengths

babynames

To modify a column, use `[]` to access the desired column, then re-assign it to a new array or Series.

In [ ]:
# Modify the "name_lengths" column to be one less than its original value.
babynames["name_lengths"] = babynames["name_lengths"] - 1
babynames

Rename a column using the `.rename()` method.

In [ ]:
# Rename "name_lengths" to "Length".
babynames = babynames.rename(columns={"name_lengths":"Length"})
babynames

Remove a column using `.drop()`.

In [ ]:
# Remove our new "Length" column.
babynames = babynames.drop("Length", axis="columns")
babynames

## Slido Exercise

Which of the following returns `'b'`?

In [ ]:
df = pd.DataFrame({
    "a": ['c', 'b', 'a'],
    "b": ['b', 'c', 'a'],
    "c": ['c', 'a', 'b']
})
df.index = ['c','b','a']
df

In [ ]:
df.drop('b')['a'][1]

In [ ]:
df[df['a'].isin(['b','c'])].iloc[0,1]

In [ ]:
df.rename(columns={'b':'a', 'a':'b'}).loc['a','b']

## Useful Utility Functions

#### `NumPy`

The `NumPy` functions you encountered in [Data 8](https://www.data8.org/su23/reference/#array-functions-and-methods) are compatible with objects in `pandas`. 

In [ ]:
yash_counts = babynames[babynames["Name"]=="Yash"]["Count"]
yash_counts

In [ ]:
# Average number of babies named Yash each year
np.mean(yash_counts)

In [ ]:
# Max number of babies named Yash born in any single year
max(yash_counts)

#### Built-In `pandas` Methods

There are many, *many* utility functions built into `pandas`, far more than we can possibly cover in lecture. You are encouraged to explore all the functionality outlined in the `pandas` [documentation](https://pandas.pydata.org/docs/reference/index.html).

In [ ]:
# Returns the shape of the object in the format (num_rows, num_columns)
babynames.shape

In [ ]:
# Returns the total number of entries in the object, equal to num_rows * num_columns
babynames.size

In [ ]:
# What summary statistics can we describe?
babynames.describe()

In [ ]:
# Our statistics are slightly different when working with a Series.
babynames["Sex"].describe()

In [ ]:
# Randomly sample row(s) from the DataFrame.
babynames.sample()

In [ ]:
# Rerun this cell a few times – you'll get different results!
babynames.sample(5).iloc[:, 2:]

In [ ]:
# Sampling with replacement.
babynames[babynames["Year"]==2000].sample(4, replace=True).iloc[:,2:]

In [ ]:
# Count the number of times each unique value occurs in a Series.
babynames["Name"].value_counts()

In [ ]:
# Return an array of all unique values in the Series.
babynames["Name"].unique()

In [ ]:
# Sort a Series. Notice that the index changes order, too!
babynames["Name"].sort_values()

In [ ]:
# Sort a DataFrame – there are lots of Michaels in California.
babynames.sort_values(by="Count", ascending=False)

## Slido Exercise

Which of the following extracts the rows of `df` where the values in column `A` are at least as big as the smallest value in column `B`?

In [ ]:
# Sample DataFrame for experimentation!
df = pd.DataFrame({
    "A": [2, 3, 4],
    "B": [3, 5, 5],
})
df

In [ ]:
df[df['A'] >= df['B'].min()]

In [ ]:
df[df['A'] >= df['B'].sort_values(ascending=False)[0]]

In [ ]:
df[df['A'] >= df['B'].value_counts().tail(n=1)]

## Custom sorting

### Approach 1: Create a temporary column

In [ ]:
# Create a Series of the length of each name.
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name.
babynames["name_lengths"] = babyname_lengths
babynames.head(5)

In [ ]:
# Sort by the temporary column.
babynames = babynames.sort_values(by="name_lengths", ascending=False)
babynames.head(5)

In [ ]:
# Drop the 'name_length' column.
babynames = babynames.drop("name_lengths", axis="columns")
babynames.head(5)

### Approach 2: Sorting using the `key` argument

In [ ]:
babynames.sort_values("Name", key=lambda x:x.str.len(), ascending=False).head()

### Approach 3: Sorting Using the `map` Function

We can also use the Python map function if we want to use an arbitrarily defined function. Suppose we want to sort by the number of occurrences of "dr" plus the number of occurences of "ea".

In [ ]:
# First, define a function to count 
# the number of times "dr" or "ea" appear in each name.
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

# Then, use `map` to apply `dr_ea_count` to each name in the "Name" column.
babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)

# Sort the DataFrame by the new "dr_ea_count" column 
# so we can see our handiwork.
babynames = babynames.sort_values(by="dr_ea_count", ascending=False)
babynames.head()

In [ ]:
# Drop the `dr_ea_count` column.
babynames = babynames.drop("dr_ea_count", axis="columns")
babynames.head(5)